In [137]:
import os
from selenium import webdriver
import time
#need to plug in the custom location of the driver for some reason.
chromedriver = "/Users/Nick/dataProjects/chromedriver"

In [138]:
driver = webdriver.Chrome(chromedriver)
driver.get("http://www.sustainabletable.org/seasonalfoodguide/")

Now we can grab all the options in the dropdown menu for state

In [139]:
#select the state dropdown
states = driver.find_element_by_id("state_select")

#grab the html of the state lists
all_states = states.find_elements_by_tag_name("option")

#store that into a vector. 
stateList = [state.get_attribute("value") for state in all_states]

#the first value is empty
stateList = stateList[1:len(stateList)] 

Let's do it again real quick for the seasons list

In [140]:
seasons     = driver.find_element_by_id("select_season")
all_seasons = seasons.find_elements_by_tag_name("option")
seasonList  = [season.get_attribute("value") for season in all_seasons]
seasonList  = seasonList[1:len(seasonList)]

First select an element in the state dropdown, then the seasons dropdown then click go and get the results. 

In [142]:
data = dict.fromkeys(stateList, None) #initialize state level of dictionary
#Scan through the states
for current_state in stateList:
    #now we initialize this states season list.
    data[current_state] = dict.fromkeys(seasonList, None)
    #Scan through the seasons
    for current_season in seasonList:
        
        #first we select the state
        for state in all_states:
            if(state.get_attribute("value") == current_state):
                state.click()

        #Next we select the season   
        for season in all_seasons:
            if(season.get_attribute("value") == current_season):
                season.click()
                
        #Click on the go button
        go = driver.find_element_by_id("get_list")
        go.click()

        #wait a tiny bit for the page to load
        time.sleep(1.2) # delays for 1 second

        #function to scrape table and return as an array of jsons
#         print(scrape_table(driver))
        data[current_state][current_season] = scrape_table()

In [128]:
def scrape_table():
    data = []
    for tr in driver.find_elements_by_xpath('//table[@id="result_table"]//tr'):
        tds = tr.find_elements_by_tag_name('td')
        if (tds): 
            if len(tds) == 3:
                data.append([td.text for td in tds])
        
    jsonified = [{"name": x[0], "type": x[1]} for x in data]
    return(jsonified)
scrape_table()

[{'name': u'Arugula', 'type': u'Vegetable'},
 {'name': u'Beets', 'type': u'Vegetable'},
 {'name': u'Broccoli', 'type': u'Vegetable'},
 {'name': u'Broccoli Rabe (Rapini)', 'type': u'Vegetable'},
 {'name': u'Brussels Sprouts', 'type': u'Vegetable'},
 {'name': u'Cabbage', 'type': u'Vegetable'},
 {'name': u'Carrots', 'type': u'Vegetable'},
 {'name': u'Celery', 'type': u'Vegetable'},
 {'name': u'Celery Root', 'type': u'Vegetable'},
 {'name': u'Clementines', 'type': u'Fruit'},
 {'name': u'Garlic', 'type': u'Vegetable'},
 {'name': u'Grapefruit, Pomelo', 'type': u'Fruit'},
 {'name': u'Greens (Collards, Etc.)', 'type': u'Vegetable'},
 {'name': u'Herbs', 'type': u'Herb'},
 {'name': u'Kohlrabi', 'type': u'Vegetable'},
 {'name': u'Leeks', 'type': u'Vegetable'},
 {'name': u'Lemons', 'type': u'Fruit'},
 {'name': u'Lettuce', 'type': u'Vegetable'},
 {'name': u'Oranges', 'type': u'Fruit'},
 {'name': u'Parsnips', 'type': u'Vegetable'},
 {'name': u'Radishes', 'type': u'Vegetable'},
 {'name': u'Rutabaga',

In [143]:
data

{u'Alabama': {u'Early April': [{'name': u'Asparagus', 'type': u'Vegetable'},
   {'name': u'Carrots', 'type': u'Vegetable'},
   {'name': u'Cauliflower', 'type': u'Vegetable'},
   {'name': u'Chard', 'type': u'Vegetable'},
   {'name': u'Fava Beans', 'type': u'Vegetable'},
   {'name': u'Fennel', 'type': u'Vegetable'},
   {'name': u'Green Onions, Scallions', 'type': u'Vegetable'},
   {'name': u'Greens (Collards, Etc.)', 'type': u'Vegetable'},
   {'name': u'Herbs', 'type': u'Herb'},
   {'name': u'Kale', 'type': u'Vegetable'},
   {'name': u'Leeks', 'type': u'Vegetable'},
   {'name': u'Lettuce', 'type': u'Vegetable'},
   {'name': u'Mushrooms', 'type': u'Vegetable'},
   {'name': u'Nettles', 'type': u'Vegetable'},
   {'name': u'Pea Shoots', 'type': u'Vegetable'},
   {'name': u'Pecans', 'type': u'Legume'},
   {'name': u'Radishes', 'type': u'Vegetable'},
   {'name': u'Rhubarb', 'type': u'Vegetable'},
   {'name': u'Spinach', 'type': u'Vegetable'},
   {'name': u'Strawberries', 'type': u'Fruit'},
   

Export all this to a nice json file to use with d3. 

In [146]:
import json
with open('seasonalData.json', 'w') as outfile:
    json.dump(data, outfile)

In [155]:
# follow the "name", "children" hierarchy
d3Data = {"name": "State", 
          "children": []}

In [156]:
for state in stateList: 
    stateData = {"name": state, "children": []}
    for season in seasonList:
        seasonChild = data[state][season] #the lowest level of child. 
        stateData["children"].append({"name": season, "children": seasonChild}) #append to season
        
    d3Data["children"].append(stateData) #now the state level data to the highest level. 

In [158]:
with open('seasonData_d3.json', 'w') as outfile:
    json.dump(d3Data, outfile)